## Análisis exploratorio de datos

### 1. Importamos las respectivas librerías

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

### 2. Creamos algunas variables de entorno para poder consultar la información dentro del data lake

In [0]:
# dl_location = 'abfss://dataengineering@datarepositoryg4jdurand1.dfs.core.windows.net/'
dl_location = 'abfss://dataengineering@bidatareposig3kquinteroc.dfs.core.windows.net/'
# dl_location = 'abfss://dataengineering@datarepositoryg4jdurand1.dfs.core.windows.net/'
raw_location = dl_location + 'RAW/'
print(raw_location)

### 3. Inspeccionamos cómo se ven los archivos del data lake

Primero exploremos cómo se ven las carpetas que cargamos en el data lake

In [0]:
dbutils.fs.ls(raw_location)

Ahora inspeccionemos algúna carpeta aleatoria, Data_5 por ejemplo

In [0]:
dbutils.fs.ls("abfss://dataengineering@datarepositoryg4jdurand1.dfs.core.windows.net/RAW/circuits.csv")

### 4. Tomamos algún archivo aleatorio para hacer una exploración de datos

Cargamos la información dentro en un dataframe de spark

In [0]:
df = (
    spark
    .read
    .format('csv')
    .option('header', True)
    .load('abfss://dataengineering@datarepositoryg4jdurand1.dfs.core.windows.net/RAW/circuits.csv')
)

display(df)

Realizamos una pequeña descripción estadística de los datos

In [0]:
display(df.describe())

Ahora vemos cuántos valores nulos hay en cada columna

In [0]:
nulos = df.select(
    [
        round(sum(col(c).isNull().cast('int'))/2708368, 2).alias(c) for c in df.columns
    ]
)

display(nulos)

Verificamos el esquema del dataframe

In [0]:
df.schema

Por último vemos cuántos valores únicos existen a lo largo de la columnas

In [0]:
dict= {}

for col in df.columns:
    distinct_count = df.select(col).distinct().count()
    dict[col] = distinct_count


display(dict)

### 5. Vamos a verificar si para los clientes que están vacíos también está vacía la segmentación

Creamos una tabla temporal para poder consultar la información utilizando SQL

In [0]:
df.createOrReplaceTempView("sample")

Caso 1: No hay código de cliente y por lo tanto no existe una segmentación asociada

In [0]:
%sql
SELECT
  COUNT(CUST_CODE) AS VALIDACION_1
FROM sample
WHERE
  CUST_CODE IS NULL 
  AND seg_1 IS NOT NULL 
  AND seg_2 IS NOT NULL
;

Caso 2: Hay un codigo de cliente pero no hay segmentación

In [0]:
%sql
SELECT
  COUNT(CUST_CODE) AS VALIDACION_1
FROM sample
WHERE
  CUST_CODE IS NOT NULL 
  AND (seg_1 IS NULL OR seg_2 IS NULL)
;

### 6. Conclusiones

- Debemos llenar los valores vacíos del código de cliente con algún valor en particular: `NO_CODE`
- La columna fecha debe convertirse a un formato adecuado.
- `QUANTITY` y `EXPEND` son las únicas columnas que deben converstirse a `INTEGER` y `DOUBLE`, respectivamente